In [1]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
#To create variables for which the the paths to the train, valid, and test data directories are assigned.

train_path = 'dataset/train'
valid_path = 'dataset/valid'
test_path = 'dataset/test'

In [8]:
# To use the Keras ImageDataGenerator class to create batches of data from the train, valid, and test directories.

train_batches = ImageDataGenerator().flow_from_directory(directory=train_path, target_size=(256,256),
    classes=['airplane', 'airport', 'baseball_diamond', 'basketball_court', 'beach', 'bridge', 'chaparral', 'church', 'circular_farmland', 'cloud', 'commercial_area', 'dense_residential', 'desert', 'forest', 'freeway'], batch_size=128)


valid_batches = ImageDataGenerator().flow_from_directory(directory=valid_path, target_size=(256,256),
    classes=['airplane', 'airport', 'baseball_diamond', 'basketball_court', 'beach', 'bridge', 'chaparral', 'church', 'circular_farmland', 'cloud', 'commercial_area', 'dense_residential', 'desert', 'forest', 'freeway'], batch_size=128)


test_batches = ImageDataGenerator().flow_from_directory(directory=test_path, target_size=(256,256),
    classes=['airplane', 'airport', 'baseball_diamond', 'basketball_court', 'beach', 'bridge', 'chaparral', 'church', 'circular_farmland', 'cloud', 'commercial_area', 'dense_residential', 'desert', 'forest', 'freeway'], batch_size=128)




Found 7500 images belonging to 15 classes.
Found 1499 images belonging to 15 classes.
Found 1501 images belonging to 15 classes.
